# Notebook setup

This notebook is intended to be used online on the Einstein Toolkit tutorial server, offline as a read-only document. To make all of these work some setting need to be tweaked, which we do in the next cell.

In [1]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)

# this (non-default package) keeps the end of shell output in view
try: import scrolldown
except ModuleNotFoundError: pass

# We are going to install kuibit, a Python package to post-process Cactus simulations.
# We will install kuibit inside the Cactus directory. The main reason for this is to
# have a make easier to uninstall kuibit (you can just remove the Cactus folder). 
import os, sys
os.environ["PYTHONUSERBASE"] = os.environ['HOME'] + "/Cactus/python"
sys.path.insert(1, f"{os.environ['PYTHONUSERBASE']}/lib/python{sys.version_info[0]}.{sys.version_info[1]}/site-packages")


Automagic is ON, % prefix IS NOT needed for line magics.


# Running a basic configuration

The configuration i want to run is the evolution of a TOV star in a cartesian grid in the einstein toolkit environment. This work is taken from my thesis, so let's access the configuration files.
First I'll show where my configuration files are

In [2]:
%%bash
cd ~/programmi/tov_ET/recon_method/
ls | grep .par

tov_ET_eno.par
tov_ET_ppm.par
tov_ET_tvd.par
tov_ET_weno.par


And now i show where my Cactus path is

In [3]:
cd ~/ET/Cactus

/home/sam/ET/Cactus


Now let's run a simple simulation for the weno reconstruction method.
I usually prefer to have my simulations folder inside the ET folder, to do that you need to edit the `basedir` value in `simfactory/etc/defs.local.ini`

``` 
[swift]
basedir = ~/ET/simulations
```

For my case i also decided to move the Cactus directory in another path, if you do so you have to tell the ET that your `sourcebasedir` has changed.
Just go to `/home/sam/ET/Cactus/simfactory/mdb/machines` and look for your machine hostname .ini file and then change the following line, in my case `swift.ini`

```
sourcebasedir   = /home/sam/ET
```

In [11]:
%%bash
./simfactory/bin/sim create tov_ET_weno --configuration sim --parfile=~/programmi/tov_ET/recon_method/tov_ET_weno.par

Parameter file: /home/sam/programmi/tov_ET/recon_method/tov_ET_weno.par
Skeleton Created
Job directory: "/home/sam/ET/simulations/tov_ET_weno"
Executable: "/home/sam/ET/Cactus/exe/cactus_sim"
Option list: "/home/sam/ET/simulations/tov_ET_weno/SIMFACTORY/cfg/OptionList"
Submit script: "/home/sam/ET/simulations/tov_ET_weno/SIMFACTORY/run/SubmitScript"
Run script: "/home/sam/ET/simulations/tov_ET_weno/SIMFACTORY/run/RunScript"
Parameter file: "/home/sam/ET/simulations/tov_ET_weno/SIMFACTORY/par/tov_ET_weno.par"


As you can see, the structure of the simulation has been created, that mean ET has allocated the resources and activated the thorns needed in order to run the simulation. One can check the structure of this by going in the directory

In [12]:
%%bash
cd ~/ET/simulations/tov_ET_weno/
ls

log.txt  SIMFACTORY


And finally now, we can start the simulation. Remember to install the packae `openmpi-bin`

In [13]:
%%bash
# start simulation segment
./simfactory/bin/sim submit tov_ET_weno --cores=2 --num-threads=1 --walltime=0:20:00

Assigned restart id: 0 
Executing submit command: exec nohup /home/sam/ET/simulations/tov_ET_weno/output-0000/SIMFACTORY/SubmitScript < /dev/null > /home/sam/ET/simulations/tov_ET_weno/output-0000/tov_ET_weno.out 2> /home/sam/ET/simulations/tov_ET_weno/output-0000/tov_ET_weno.err & echo $!
Submit finished, job id is 730293


The `submit` command submitted a new segment for the simulation `tov_ET_weno` to the queueing system to run in the background asking for a maximum runtime of 20 minutes, using a total of 2 compute cores and using 1 thread per MPI ranks. On your laptop it will start right away, on a cluster the queuing system will wait until a sufficient number of nodes is able to start your simulation.

You can check the status of the simulation with the following command

In [14]:
%%bash
# watch log output, following along as new output is produced
./simfactory/bin/sim show-output --follow tov_ET_weno

Simulation name: tov_ET_weno
==> tov_ET_weno.out <==
   ADMBASE::gxy
   ADMBASE::gxz
   ADMBASE::gyy
   ADMBASE::gyz
   ADMBASE::gzz
   ADMBASE::kxx
   ADMBASE::kxy
   ADMBASE::kxz
   ADMBASE::kyy
   ADMBASE::kyz
   ADMBASE::kzz
   ADMBASE::alp
   HYDROBASE::rho
   HYDROBASE::press
   HYDROBASE::eps
   HYDROBASE::vel[0]
   HYDROBASE::vel[1]
   HYDROBASE::vel[2]
   ML_BSSN::H
   ML_BSSN::M1
   ML_BSSN::M2
   ML_BSSN::M3
INFO (CarpetIOASCII): I/O Method 'IOASCII_2D' registered: 2D AMR output of grid variables to ASCII files
INFO (CarpetIOASCII): I/O Method 'IOASCII_3D' registered: 3D AMR output of grid variables to ASCII files
INFO (CarpetIOHDF5): I/O Method 'IOHDF5' registered: AMR output of grid variables to HDF5 files
INFO (CarpetIOHDF5): I/O Method 'IOHDF5_0D' registered: 0D AMR output of grid variables to HDF5 files
INFO (CarpetIOHDF5): I/O Method 'IOHDF5_1D' registered: 1D AMR output of grid variables to HDF5 files
INFO (CarpetIOHDF5): I/O Method 'IOHDF5_2D' registered: 2D AMR outp

INFO (CartGrid3D): Grid Spacings:
INFO (CartGrid3D): dx=>2.0000000e+00  dy=>2.0000000e+00  dz=>2.0000000e+00
INFO (CartGrid3D): Computational Coordinates:
INFO (CartGrid3D): x=>[-6.000,28.000]  y=>[-6.000,28.000]  z=>[-6.000,28.000]
INFO (CartGrid3D): Indices of Physical Coordinates:
INFO (CartGrid3D): x=>[0,17]  y=>[0,17]  z=>[0,17]
INFO (TerminationTrigger): Reminding you every 60 minutes about remaining walltime
INFO (Time): Timestep set to 0.5 (courant_static)
INFO (GRHydro): Trying to get EOS handles
INFO (GRHydro): Trying to get EOS handles
INFO (GRHydro): GRHydro will use the 2D_Polytrope equation of state.
INFO (GRHydro): Setting up the atmosphere mask: all points are not_atmosphere
INFO (TOVSolver): Integrated TOV equation
INFO (TOVSolver): Information about the TOVs used:
INFO (): TOV    radius    mass  bary_mass mass(g) cent.rho rho(cgi)        K   K(cgi)    Gamma
INFO ():   1   8.12502  1.40016  1.50618 2.78e+33  0.00128 7.92e+14      100 1.45e+05        2
INFO (TOVSolver):

To abort the simulation just run

In [ ]:
%%bash
./simfactory/bin/sim stop tov_ET_weno

To list all simulations kept in the cache

In [16]:
%%bash
./simfactory/bin/sim list-simulations

   tov_ET_weno             [ACTIVE (RUNNING), restart 0000, job id 730293]


The output will be stored in a directory, to get its position

In [9]:
%%bash
./simfactory/bin/sim get-output-dir tov_ET_weno

/home/sam/ET/simulations/tov_ET_weno/output-0000


If you're done with your simulation just execute

In [10]:
%%bash
./simfactory/bin/sim purge tov_ET_weno

Simulation name: tov_ET_weno
Simulation tov_ET_weno has been moved to trash folder /home/sam/ET/simulations/TRASH/simulation-tov_ET_weno-swift-swift-sam-2023.03.01-22.25.19-729866


# Plotting the results

We care to plot the results of the central density evolution with respect to time to show the oscillating behaviour of the perturbed star